# Two-Phase Flash Calculation (VLE and LLE)

The isothermal-isobaric two-phase flash is carried out by a combination of accelerated successive substitution (ASS) to update phase compositions and Halley's method to solve the Rachford-Rice mass balance. 

$$ FO = \sum_{i=1}^c \left( x_i^\beta - x_i^\alpha \right) = \sum_{i=1}^c \frac{z_i (K_i-1)}{1+\psi (K_i-1)} $$

Where,  $K_i = x_i^\beta/x_i^\alpha =\hat{\phi}_i^\alpha /\hat{\phi}_i^\beta  $ represents the equilibrium constant  and $\psi$ the fraction of the phase $\beta$. The described method can be slow at high pressures, for that reason, the number of cycles of ASS is limited to ``nacc`` cycles and if no solution is reached the algorithm changes to a second-order procedure based on Gibbs free energy minimization:

$$ min \, {G(\underline{F}^\alpha, \underline{F}^\beta)} = \sum_{i=1}^c (F_i^\alpha \ln \hat{f}_i^\alpha + F_i^\beta \ln \hat{f}_i^\beta) $$

Here, $F$ refers to the number of moles and $\hat{f}$ to the effective fugacity, the superscript refers to the phase index and the subscript to the specie index. The optimization is performed using SciPy minimization routines.

Phasepy ``flash`` routine solves phase compositions for two-phase flash at constant pressure and temperature (PT-flash).  The ```flash``` function requires initial guesses for the phase compositions, the states of the phases, i.e ```LV``` fo liquid-vapor flash or ```LL``` for liquid-liquid flash, the global phase composition (``z``),  the temperature (``T``) and pressure (``P``) of the system.

In this notebook, Vapor-Liquid and Liquid-Liquid flash calculation will be exemplified. Examples below apply Peng-Robinson equation of state. The mixing rule applied is Modified Huron Vidal combined with Modified UNIFAC.

To start, the required functions are imported.

In [4]:
import numpy as np
from phasepy import mixture, component, preos
from phasepy.equilibrium import flash

---
### Vapor Liquid Equilibrium Flash

This calculation will be exemplified for the mixture of ethanol and water.

In [5]:
water = component(name='water', Tc=647.13, Pc=220.55, 
                  w=0.344861, GC={'H2O':1})

ethanol = component(name='ethanol', Tc=514.0, Pc=61.37,
                    w=0.643558, GC={'CH3':1, 'CH2':1, 'OH(P)':1})

mix = mixture(ethanol, water)
# or
mix = ethanol + water

mix.unifac() 
eos = preos(mix, 'mhv_unifac')
T = 350.0
P = 0.8
Z = np.array([0.4, 0.6])
x0 = np.array([0.23512692, 0.76487308])
y0 = np.array([0.5, 0.5])
flash(x0, y0, 'LV', Z, T, P, eos, full_output=True)

      T: 350.0
      P: 0.8
   beta: 0.46657039791747923
  error: 9.45706631675112e-09
   iter: 7
      X: array([0.23951597, 0.76048403])
     v1: 32.50887949066499
 state1: 'L'
      Y: array([0.58348128, 0.41651872])
     v2: 35869.13850298779
 state2: 'V'

---
### Liquid Liquid Equilibrium Flash

This calculation will be exemplified for the mixture of water and butanol.

In [6]:
water = component(name='water', Tc=647.13, Pc=220.55, Zc=0.229, Vc=55.948, w=0.344861,
                  Ant=[11.64785144, 3797.41566067, -46.77830444],
                  GC={'H2O':1})

mtbe = component(name='mtbe', Tc=497.1, Pc=34.3, Zc=0.273, Vc=329.0, w=0.266059,
                 Ant=[9.16238246, 2541.97883529, -50.40534341],
                 GC={'CH3':3, 'CH3O':1, 'C':1})

mix = mixture(water, mtbe)
# or
mix = water + mtbe
mix.unifac()
eos = preos(mix, 'mhv_unifac')
T = 320.0
P = 1.01
Z = np.array([0.5, 0.5])
x0 = np.array([0.01, 0.99])
w0 = np.array([0.99, 0.01])
flash(x0, w0, 'LL', Z, T, P, eos, full_output=True)

      T: 320.0
      P: 1.01
   beta: 0.46991089151358084
  error: 2.9082881326185033e-09
   iter: 7
      X: array([0.05876434, 0.94123566])
     v1: 112.09263402948598
 state1: 'L'
      Y: array([0.99774164, 0.00225836])
     v2: 21.756817287953805
 state2: 'L'

For further information please also check [official documentation](https://phasepy.readthedocs.io/), or just try:

```function?```